In [ ]:
import openai
import requests
import json
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# API keys for OpenAI (ChatGPT) and Google Gemini
OPENAI_API_KEY = "xx"
GEMINI_API_KEY = "xx"

# Set up OpenAI API
openai.api_key = OPENAI_API_KEY

# Load your dataset of textual samples (assumed to be a CSV)
df = pd.read_csv('text_samples.csv')  # Replace with actual file path
texts = df['text'].tolist()
labels = df['label'].tolist()  # Assuming 1 for positive, 0 for negative

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

def preprocess_text(text):
    # preprocessing: lowercasing and removing non-alphanumeric characters
    return ''.join([char.lower() if char.isalnum() or char.isspace() else ' ' for char in text])

# Preprocess all texts
train_texts = [preprocess_text(text) for text in train_texts]
test_texts = [preprocess_text(text) for text in test_texts]


def classify_with_chatgpt(text, few_shot_examples=None):

    prompt = f"Classify the sentiment of the following text as Positive, Negative, or Neutral:\n\n{text}"

    # Adding few-shot examples to the prompt if available
    if few_shot_examples:
        prompt = "Here are a few examples to guide sentiment classification:\n\n" + few_shot_examples + "\n\n" + prompt

    try:
        response = openai.Completion.create(
            model="gpt-4",
            prompt=prompt,
            max_tokens=10,
            temperature=0
        )
        result = response['choices'][0]['text'].strip().lower()
        if 'positive' in result:
            return 1  # Positive
        elif 'negative' in result:
            return 0  # Negative
        else:
            return 2  # Neutral
    except Exception as e:
        print(f"Error with ChatGPT: {e}")
        return 2  #neutral in case of an error

def classify_with_gemini(text, few_shot_examples=None):
    try:
        headers = {"Authorization": f"Bearer {GEMINI_API_KEY}", "Content-Type": "application/json"}
        data = {
            "text": text,
            "model": "gemini-llm",  # Hypothetical parameter for Google Gemini's LLM API
            "few_shot_examples": few_shot_examples if few_shot_examples else ""
        }
        response = requests.post("https://gemini-api-url/sentiment", headers=headers, json=data)
        result = response.json().get("sentiment", "neutral").lower()
        if 'positive' in result:
            return 1  # Positive
        elif 'negative' in result:
            return 0  # Negative
        else:
            return 2  # Neutral
    except Exception as e:
        print(f"Error with Gemini: {e}")
        return 2  # Default to neutral in case of an error

def evaluate_model(predictions, true_labels):

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")
    return accuracy, precision, recall, f1



# Create a few-shot learning prompt from the training set
few_shot_examples = "\n".join([f"Text: {train_texts[i]}\nLabel: {'Positive' if train_labels[i] == 1 else 'Negative'}"
                                for i in range(3)])  # Using the first 3 examples

def zero_shot_classification(texts, classifier_func):
    predictions = [classifier_func(text) for text in texts]
    return predictions

def few_shot_classification(test_texts, classifier_func, few_shot_examples):
    predictions = [classifier_func(text, few_shot_examples) for text in test_texts]
    return predictions


# Perform Zero-Shot Sentiment Analysis with ChatGPT
print("Zero-Shot Classification with ChatGPT")
chatgpt_predictions = zero_shot_classification(test_texts, classify_with_chatgpt)
evaluate_model(chatgpt_predictions, test_labels)

# Perform Few-Shot Sentiment Analysis with ChatGPT
print("\nFew-Shot Classification with ChatGPT")
chatgpt_few_shot_predictions = few_shot_classification(test_texts, classify_with_chatgpt, few_shot_examples)
evaluate_model(chatgpt_few_shot_predictions, test_labels)

# Perform Zero-Shot Sentiment Analysis with Google Gemini
print("\nZero-Shot Classification with Google Gemini")
gemini_predictions = zero_shot_classification(test_texts, classify_with_gemini)
evaluate_model(gemini_predictions, test_labels)

# Perform Few-Shot Sentiment Analysis with Google Gemini
print("\nFew-Shot Classification with Google Gemini")
gemini_few_shot_predictions = few_shot_classification(test_texts, classify_with_gemini, few_shot_examples)
evaluate_model(gemini_few_shot_predictions, test_labels)



start_time = time.time()
for _ in range(10_000):
    classify_with_chatgpt("Sample text for processing speed assessment.")
end_time = time.time()

print(f"\nProcessed 10,000 samples in {end_time - start_time:.2f} seconds.")
